# Classes

In [ ]:
class Player:
    
    '''
    Base class used to instantiate the dealer.
    '''
    
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return f'{self.name}'

In [ ]:
class HumanPlayer(Player):
    
    '''
    Class derived from base class Player. Additional attribute for bankroll balance for the bets placed by the human player.
    '''
    
    def __init__(self, name, bankroll):
        Player.__init__(self, name)
        self.bankroll = bankroll
        print(f'Your bankroll balance is {self.bankroll}.')
        
    def __str__(self):
        return f'Player: {self.name}, Bankroll Balance: {self.bankroll}.'

In [ ]:
class Card:
    
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        
    def __str__(self):
        return self.rank + ' of ' + self.suit

In [ ]:
class Deck:
    
    def __init__(self):
        self.deck = []
        for suit in suits_tpl:
            for rank in ranks_tpl:
                self.deck.append(Card(rank, suit))
    
    def __str__(self):
        deck_str = ''
        for card in self.deck:
            deck_str += '\n' + card.__str__()
        return 'The deck consists of: '+ deck_str
    
    def shuffle(self):
        random.shuffle(self.deck)
    
    def deal(self):
        return self.deck.pop()

In [ ]:
class Hand:
    
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0
                
    def __str__(self):
        hand_str = ''
        for card in self.cards:
            hand_str += '\n' + card.__str__()
        hand_str += f'\nThe total hand value is {self.value}'
        return hand_str
    
    def add_card(self, card):
        self.cards.append(card)
        self.value += values_dct[card.rank]
        
        if card.rank == 'Ace':
            self.aces += 1
    
    def initial_deal(self):
        for i in range(1,3):
            self.add_card(deck.deal())
            
    def adjust_for_aces(self):
        if self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1
            
    def check_value(self):
        while self.value > 21 and self.aces:
            self.adjust_for_aces()
            print('Adjusted for an ace!')
            
        if self.value == 21:
            print('Black Jack!')
            return False
        elif self.value > 21:
            print('Bust!')
            return False
        else:
            return True

# Game functions

In [ ]:
def get_bet_value(player):
    while True:
        try:
            bet_value = int(input(f'Your current balance is: {player.bankroll}. How much do you want to bet?: '))
        except:
            print('Invalid value entered.')
        else:
            if bet_value > player1.bankroll:
                print('You don\'t have sufficient credit. Enter a lower value.')
            else:
                break

    return bet_value

In [ ]:
def compare_player_dealer_hand(player_hand, dealer_hand):
    if player_hand.value <= dealer_hand.value:
        print('The dealer wins!')
        return False
    else:
        return True

In [ ]:
def player_wins(player, bet_value):
    player.bankroll += bet_value
    print(f'You won. You receive double your bet. Your current balance is {player.bankroll}.')

In [ ]:
def dealer_wins(player, bet_value):
    player1.bankroll -= bet_value
    print(f'You lost your bet. Your current balance is {player.bankroll}.')

# Game logic

In [ ]:
import random

suits_tpl = ('Hearts', 'Diamonds', 'Clubs', 'Spades')
ranks_tpl = ('2', '3', '4', '5', '6', '7', '8', '9', '10',
             'Jack', 'Queen', 'King', 'Ace')
values_dct = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6,
              '7': 7, '8': 8, '9': 9, '10': 10, 'Jack': 10,
              'Queen': 10, 'King': 10, 'Ace': 11}

game_on = True

player_name = input('What is your name: ')
player1 = HumanPlayer(player_name, 100)
dealer = Player('Dealer')

while game_on: # game
    
    win = False
    
    player1_hand = Hand()
    dealer_hand = Hand()
    
    deck = Deck()
    deck.shuffle()

    # player places bet
    bet_value = get_bet_value(player1)
    
    # player gets two cards, cards are displayed
    player1_hand.initial_deal()
    
    while game_on: # player's turn

        #check player hand's value
        game_on = player1_hand.check_value()
        
        if not game_on:
            break

        print(f'\nYou hold {player1_hand.__str__()}.')
        choice = input('Do you want to hit or stand? ')

        if choice == 'stand':
            break
        elif choice == 'hit':
            dealt_card = deck.deal()
            player1_hand.add_card(dealt_card)
            print(f'You were dealt {dealt_card.__str__()}.')
        else:
            print('Invalid input. Please enter hit or stand.')

    if player1_hand.value > 21:
        dealer_wins(player1, bet_value)
    elif player1_hand.value <= 21: # dealer's turn
        
        print("\nNow it's the dealer's turn.")

        # dealer gets two cards, cards are displayed
        dealer_hand.initial_deal()

        game_on = True

        while game_on:
            
            print(f'\nThe dealer holds {dealer_hand.__str__()}.')
            
            # dealer hits until exceeds player score <21, hits 21, or goes bust
            game_on = dealer_hand.check_value()
            
            if not game_on and dealer_hand.value != 21:
                player_wins(player1, bet_value)
                break

            game_on = compare_player_dealer_hand(player1_hand, dealer_hand)

            if not game_on:
                dealer_wins(player1, bet_value)
                break
            
            dealt_card = deck.deal()
            dealer_hand.add_card(dealt_card)
            print(f'The dealer was dealt {dealt_card.__str__()}.')

    # player can decide to play again or end the game
    if player1.bankroll:
        while True:
            choice = input('Would you like to play another round (yes/no)?: ')
            if choice == 'yes':
                game_on = True
                break
            elif choice == 'no':
                print('Game over.')
                game_on = False
                break
            else:
                print('Invalid input. Please enter yes or no.')
    else:
        print('You are out of funds. Game over.')
        break